In [2]:
import pandas as pd
import os
from pathlib import Path

# === PART 1: Read and consolidate data ===
input_folder = Path("input_files")
output_file = Path("output/smart_dashboard.xlsx")
output_file.parent.mkdir(exist_ok=True)

all_data = []

for file in input_folder.glob("*.xlsx"):
    try:
        df = pd.read_excel(file, sheet_name="Sales", engine='openpyxl')
        all_data.append(df)
    except Exception as e:
        print(f"❌ Skipping {file.name}: {e}")

data = pd.concat(all_data, ignore_index=True)
data['Date'] = pd.to_datetime(data['Date'])
data['Month'] = data['Date'].dt.to_period('M').astype(str)

# === PART 2: Generate KPIs ===
summary = (
    data.groupby('Month').agg(
        Total_Revenue=('Revenue', 'sum'),
        Total_Orders=('Revenue', 'count'),
        Avg_Order_Value=('Revenue', 'mean'),
        Top_Product=('Product', lambda x: x.value_counts().idxmax()),
        Top_Branch=('Branch', lambda x: x.value_counts().idxmax())
    ).reset_index()
)

# === PART 3: Pivot-like summary ===
pivot_table = pd.pivot_table(
    data,
    index='Month',
    columns='Branch',
    values='Revenue',
    aggfunc='sum',
    fill_value=0
)

# === PART 4: Export to Excel ===
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    data.to_excel(writer, sheet_name="Raw_Data", index=False)
    summary.to_excel(writer, sheet_name="KPI_Summary", index=False)
    pivot_table.to_excel(writer, sheet_name="Pivot_Like_Summary")

    # Chart setup
    workbook = writer.book
    worksheet = writer.sheets['KPI_Summary']

    chart = workbook.add_chart({'type': 'line'})
    chart.add_series({
        'name': 'Total Revenue',
        'categories': ['KPI_Summary', 1, 0, len(summary), 0],
        'values':     ['KPI_Summary', 1, 1, len(summary), 1],
    })
    chart.set_title({'name': 'Monthly Revenue'})
    chart.set_y_axis({'name': 'Revenue'})
    worksheet.insert_chart('H2', chart)

print("✅ Excel dashboard created successfully.")


✅ Excel dashboard created successfully.
